In [38]:
import numpy as np
import sympy as sp

In [39]:
sp.symbols('v0')

v0

In [40]:
symbols = {str(key): sp.symbols(key) for key in ["v0", "w0", "E", "h", "D", "x0", "p0"]}

In [41]:
symbols

{'v0': v0, 'w0': w0, 'E': E, 'h': h, 'D': D, 'x0': x0, 'p0': p0}

In [42]:
list(symbols.keys())

['v0', 'w0', 'E', 'h', 'D', 'x0', 'p0']

In [43]:
v0, w0, E, h, D, x0, p0 = sp.symbols("v0 w0 E h D^* x0 p0")
R = sp.symbols("R")

In [44]:
coeff_list = {'membrane': v0**2 / (E * h * x0**2),
              'bending': D * w0**2 / x0**2,
              'coupling': v0 * w0 **2 / x0**2,
              'load_transverse': p0 * x0 **2 * w0,
              'load_disclinations': x0**2 * v0 * E * h}

In [45]:
coeff_list

{'membrane': v0**2/(E*h*x0**2),
 'bending': D^**w0**2/x0**2,
 'coupling': v0*w0**2/x0**2,
 'load_transverse': p0*w0*x0**2,
 'load_disclinations': E*h*v0*x0**2}

In [56]:
def apply_conditions(coeff_list):
    """Apply the conditions and solve for the parameters."""
    # Set x0 to 1
    x0_value = R
    D_value = E * h**3
    
    # Substitute x0 and D into the coefficients
    coeff_list_substituted = {key: coeff.subs({x0: x0_value, D: D_value}) 
                              for key, coeff in coeff_list.items()}
        
    return coeff_list_substituted

def rescale_by_membrane(coeff_list):
    """Rescale the coefficients by the membrane coefficient."""
    membrane_coeff = coeff_list['membrane']
    rescaled_coefficients = {key: sp.simplify(coeff / membrane_coeff) for key, coeff in coeff_list.items()}
    return rescaled_coefficients


def scaling(coeff_list):
    """Apply the conditions and solve for the parameters."""
    # Set x0 to 1
    x0_value = 1
    
    # Substitute x0 and D into the coefficients
    coeff_list_substituted = {key: coeff.subs({x0: R}) for key, coeff in coeff_list.items()}
    
    # Setting membrane, bending, and coupling coefficients to 1
    # membrane_condition = sp.Eq(coeff_list_substituted['membrane'], 1)
    coupling_condition = sp.Eq(coeff_list['coupling'], 1)
    bending_condition = sp.Eq(coeff_list['bending'], 1)
    
    # Solve for v0, w0
    v0_scale = sp.solve(bending_condition, v0)[0]
    w0_scale = sp.solve(coupling_condition.subs(v0, v0_scale), w0)[0]
    
    return v0_scale, w0_scale, coeff_list_substituted

In [57]:
coeff_list_substituted = apply_conditions(coeff_list)
coeff_list_substituted

{'membrane': v0**2/(E*R**2*h),
 'bending': E*h**3*w0**2/R**2,
 'coupling': v0*w0**2/R**2,
 'load_transverse': R**2*p0*w0,
 'load_disclinations': E*R**2*h*v0}

In [58]:
rescaled_coefficients = rescale_by_membrane(coeff_list_substituted)
rescaled_coefficients

{'membrane': 1,
 'bending': E**2*h**4*w0**2/v0**2,
 'coupling': E*h*w0**2/v0,
 'load_transverse': E*R**4*h*p0*w0/v0**2,
 'load_disclinations': E**2*R**4*h**2/v0}

In [59]:
# Pretty print the rescaled coefficients
print("Rescaled coefficients by the membrane coefficient:")
for key, value in rescaled_coefficients.items():
    print(f"{key}:")
    sp.pprint(value)
    print()  # for better readability


Rescaled coefficients by the membrane coefficient:
membrane:
1

bending:
 2  4   2
E ⋅h ⋅w₀ 
─────────
     2   
   v₀    

coupling:
      2
E⋅h⋅w₀ 
───────
   v₀  

load_transverse:
   4        
E⋅R ⋅h⋅p₀⋅w₀
────────────
      2     
    v₀      

load_disclinations:
 2  4  2
E ⋅R ⋅h 
────────
   v₀   



In [60]:
coupling_condition = sp.Eq(rescaled_coefficients['coupling'], 1)
bending_condition = sp.Eq(rescaled_coefficients['bending'], 1)

coupling_condition


Eq(E*h*w0**2/v0, 1)

In [61]:
bending_condition

Eq(E**2*h**4*w0**2/v0**2, 1)

In [62]:
v0_scaled = sp.solve(coupling_condition, v0)[0]
v0_scaled

E*h*w0**2

In [63]:
bending_condition.subs(v0, v0_scaled)

Eq(h**2/w0**2, 1)

In [64]:
w0_scaled = sp.solve(bending_condition.subs(v0, v0_scaled), w0)[1]
w0_scaled


h

In [65]:
v0_scaled = v0_scaled.subs(w0, w0_scaled)


In [68]:
coeff_list_scaled = {key: coeff.subs({v0: v0_scaled, w0: w0_scaled, x0: R}) 
                            for key, coeff in rescaled_coefficients.items()}
coeff_list_scaled

{'membrane': 1,
 'bending': 1,
 'coupling': 1,
 'load_transverse': R**4*p0/(E*h**4),
 'load_disclinations': E*R**4/h}

In [69]:
print("Rescaled coefficients by the membrane coefficient:")
for key, value in coeff_list_scaled.items():
    print(f"{key}:")
    sp.pprint(value)
    print()  # for better readability


Rescaled coefficients by the membrane coefficient:
membrane:
1

bending:
1

coupling:
1

load_transverse:
 4   
R ⋅p₀
─────
    4
 E⋅h 

load_disclinations:
   4
E⋅R 
────
 h  

